In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Pandas show more columns
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
tr = pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/train.csv')
te = pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/test.csv')

In [ ]:
tr.info()

In [ ]:
te.info()

# Interesting - every feature is an integer, and no missing values.

In [ ]:
te['target'] = -1

tr_te = pd.concat([tr, te],axis=0,sort=True).reset_index(drop=True)
tr_te['is_train'] = tr_te['target'] == -1

In [ ]:
tr_te

In [ ]:
from tqdm import tqdm

In [ ]:
# I have no idea what each feature means - let's get some descriptive meanings
list_of_dicts = []
for column_number in tqdm(range(0, 50)):
    column_name = 'feature_' + str(column_number) # Alternatively, f'feature_{column_number}'
    nunique = tr_te[column_name].nunique()
    the_min = tr_te[column_name].min()
    the_max = tr_te[column_name].max()
    the_mean = tr_te[column_name].mean()
    the_median = tr_te[column_name].median()
    the_std = tr_te[column_name].std()
    
    list_of_dicts.append(
        {'column':column_name,
         'nunique':nunique,
         'min':the_min,
         'max':the_max,
         'mean':the_mean,
         'med':the_median,
         'std':the_std
        })

In [ ]:
info = pd.DataFrame(list_of_dicts)

In [ ]:
info

# Here's my initial reaction to this:
1. Some columns can take on negative numbers, but most start at 0
2. The "nunique" is USUALLY (max - min) + 1... since these are integer columns, it makes me believe that the columns are all LabelEncoded values of categories
3. If that's the case, we should check and see if any of the features are ordinal

# Let's check if any of the features are ordinal:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = (10, 6) # Make plots bigger

In [ ]:
for column_number in tqdm(range(0, 50)):
    column_name = 'feature_' + str(column_number) # Alternatively, f'feature_{column_number}'
    ax = sns.histplot(data=tr, x=column_name, hue='target', binwidth=1)
    ax.set_title(column_name)
    plt.show() # Force it to show us the plot

# I don't think that really answered my question - I should redesign the plots such that for each unique value in the feature, we see the % allocated to each class.

In [ ]:
for column_number in tqdm(range(0, 50)):
    column_name = 'feature_' + str(column_number) # Alternatively, f'feature_{column_number}'
    df = tr.groupby(column_name)['target'].value_counts(normalize=True).to_frame('pct').reset_index()
    ax = sns.histplot(data=df, x=column_name, hue='target', bins = df[column_name].nunique(), multiple='stack', weights='pct', binrange=[df[column_name].min(),df[column_name].max()])
    ax.set_title(column_name)
    plt.show() # Force it to show us the plot

In [ ]:
# Potentially ordinal features?
# feature_48
# feature_35
# feature_34
# feature_32
# feature_30
# feature_26